# Data Cleanup


To do: 
- [ ] Field cleanup
- [ ] Standard text preprocessing

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_json('raw_data.json')
df.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",Junmai Daiginjo,30.00,Yamaguchi,+3,1.4,Chunky,Pinot Noir / Chewy Whites,Ales,"Mushroom risotto, tempura, fried chicken, cavi...",This sake has an older brother Daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",Junmai Ginjo,33.00,Niigata,+1,1.6,Citrus,Sharp Whites/Dry Reds,Ales/Ambers,Suitable for light and stronger flavored dishe...,True Sake owner Beau Timken recorded K...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",Junmai Daiginjo,49.00,Shizuoka,+/-0,1.4,Popular,Elegant reds/Complex whites,Crisp Ales,Cuisines that like big white wines.,Talk about a nice aroma package on thi...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",Tokubetsu Junmai,9.00,Yamanashi,+3,1.6,Smooth,Pinot Noir/Slippery Whites,Gentle Ales,None,The nose on this awesome looking “catch cup” i...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",Honjozo,40.00,Hyogo,±0.5,1.5,Rich,Deep Reds/Fat Whites,Stouts,Perfect for pub food salty and savory and grea...,The nose on this extremely famous Honjozo from...


First, I'm curious to see how well the description parsing worked: 

In [3]:
df.kw_word.unique()

array(['Chunky ', 'Citrus ', 'Popular ', 'Smooth ', 'Rich ', 'Even ',
       None, 'Hops ', 'Consistent ', 'Big ', 'Pear ', 'Peach ', 'Velvet ',
       '23% ', 'Vivacious ', 'Full Monty ', 'Dry      ', 'Gentle ',
       'Light ', 'Cedar ', 'Strawberry ',
       "Superb WINE: Zesty Dry Reds/Crisp Whites BEER: Crisp Ales. FOODS: The kitchen sink! Super food friendly. Updated 2019/02\xa0Vivid, voluptuous and vibrant sake in a can? You bet! This exclusive unpasteurized brew is Beau Timken's prized sake offering for True Sake customers. With a nose of strawberry, watermelon, sweet rice and cinnamon this undiluted sake (18.5%) is fat, happy, bright and chewy. Fruity, but not overly sweet, think big thoughts when tasting this velvety Nama that is pure pleasure in a can. (Yes cans keep sake colder longer!) This abundantly flavored sake is available only at True Sake! WORD: Jolly ",
       'Clean ', 'Chewy', 'Big Clean ', 'Dry ', 'Milky ', 'Beefy ',
       'Amber ', 'Flavorful ', 'Floral\xa0', 

Looks like I have a few listings where it didn't work so well... let's take a look: 

In [4]:
df[df['kw_word'].str.len() > 40]

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
23,https://www.truesake.com/collections/all/produ...,"Narutotai Ginjo Nama Genshu ""Red Snapper""",Ginjo Nama Genshu,38.00,Tokushima,+5,1.7,Superb WINE: Zesty Dry Reds/Crisp Whites BEER:...,Zesty Dry Reds/Crisp Whites BEER: Crisp Ales. ...,Crisp Ales. FOODS: The kitchen sink! Super foo...,The kitchen sink! Super food friendly. Updated...,We used to refer to this wonderful Can as “Bea...
123,https://www.truesake.com/collections/all/produ...,"Masumi Arabashiri Junmai Ginjo Nama ""First Run""",Junmai Ginjo Nama,urrent price $25.00,Nagano,-0.6,1.7,Lively WINE: Tannin Reds/ Crisp Whites BEER: C...,Tannin Reds/ Crisp Whites BEER: Citrus Ales FO...,"Citrus Ales FOODS: International Spice, Champa...","International Spice, Champagne Fare, Grilled f...",The nose on this very famous seasonally releas...
165,https://www.truesake.com/collections/all/produ...,"Harushika Shiboribana Junmai Ginjo Nama ""Sprin...",Junmai Ginjo Nama,36.00,Nara,-2,1.8,Elegant WINE: Soft Reds/Clean Whites BEER: Lig...,Soft Reds/Clean Whites BEER: Light Beer FOODS:...,"Light Beer FOODS: Clean cuisines, grilled fare...","Clean cuisines, grilled fare, shellfish, sashi...",The nose on this just released unpasteurized s...
194,https://www.truesake.com/collections/all/produ...,"Masumi Junmai Ginjo Hiyaoroshi ""Sleeping Beauty""",Junmai Ginjo Hiyaoroshi,41.00,Nagano,±0,1.8,Balanced WINE: Rich Reds/Fat Whites BEER: Belg...,Rich Reds/Fat Whites BEER: Belgian Ales FOODS:...,Belgian Ales FOODS: Fall cooked meats and fowl...,"Fall cooked meats and fowl, grilled fish, smok...",The nose on this seasonally released “Draft Sa...
203,https://www.truesake.com/collections/all/produ...,Nechi Otokoyama Tokubetsu Honjozo Nama “Yukimi...,Tokubetsu Honjozo Nama,urrent price $32.00,Niigata,+4,1.8,Clean WINE: Soft Reds/ Dry Whites BEER: Light ...,Soft Reds/ Dry Whites BEER: Light Beer FOODS: ...,"Light Beer FOODS: Salty, oily, savory grilled ...","Salty, oily, savory grilled fare, fried foods,...",The nose on this seasonally released unpasteur...
214,https://www.truesake.com/collections/all/produ...,Shirataki Jozen Junmai Ginjo Hiyaoroshi,Junmai Ginjo Hiyaoroshi,42.00,Niigata,±0,1.5,Rich WINE: Deep Reds/Fat Whites BEER: Fruity A...,Deep Reds/Fat Whites BEER: Fruity Ales FOODS: ...,Fruity Ales FOODS: All the wonderful full-bodi...,All the wonderful full-bodied and complex autu...,The nose on this seasonally released single-pa...
219,https://www.truesake.com/collections/all/produ...,Shichida Hiyaoroshi Junmai,Hiyaoroshi Junmai,44.00,Saga,+1,1.6,Berry Pie WINE: Deep Reds/Fruity Whites BEER: ...,Deep Reds/Fruity Whites BEER: Fruity Ales FOOD...,Fruity Ales FOODS: Rich and full-bodied Autumn...,"Rich and full-bodied Autumn cuisines, fatty fi...",The nose on this Fall Draft seasonally release...


In [5]:
for link in df[df['kw_word'].str.len() > 40].url:
    print(link)

https://www.truesake.com/collections/all/products/narutotai-nama-genshu
https://www.truesake.com/collections/all/products/masumi-arabashiri
https://www.truesake.com/collections/all/products/harushika-shiboribana
https://www.truesake.com/collections/all/products/masumi-junmai-ginjo-hiyaoroshi-sleeping-beauty
https://www.truesake.com/collections/all/products/nechi-otokoyama-tokubetsu-honjozo-nama-yukimizake
https://www.truesake.com/collections/all/products/shirataki-jozen-junmai-ginjo-hiyaoroshi
https://www.truesake.com/collections/all/products/shichida-hiyaoroshi-omachi-junmai


After reviewing these links, these products have revised descriptions. Basically duplicating the description section, including the 4 keywords. 

I'll do some data cleanup for these 7 products.

In [6]:
def split_on_chars(chars, text):
    if text is not None:
        if chars in text:
            return text.split(chars)[0].strip()
        else:
            return text.strip()
    else:
        return text

In [7]:
df['kw_word'] = df['kw_word'].apply(lambda description: split_on_chars('WINE', description))

In [8]:
#df.kw_word.unique()

Next - `kw_wines` column:

In [9]:
df['kw_wines'] = df['kw_wines'].apply(lambda description: split_on_chars('BEER', description))

In [10]:
#df.kw_wines.unique()

Onto `kw_beer` column:

In [11]:
df['kw_beer'] = df['kw_beer'].apply(lambda description: split_on_chars('FOOD', description))

In [12]:
#df.kw_beer.unique()

In [13]:
df['kw_beer'] = df['kw_beer'].apply(lambda description: split_on_chars('BEER:', description))
df['kw_beer'] = df['kw_beer'].apply(lambda description: split_on_chars('Foods:', description))

In [14]:
#df.kw_beer.unique()

Onto `kw_foods` column: 

In [15]:
#df.kw_foods.unique()

In [16]:
df['kw_foods'] = df['kw_foods'].apply(lambda description: split_on_chars('Updated', description))
df['kw_foods'] = df['kw_foods'].apply(lambda description: split_on_chars('(', description))

In [17]:
df.prefecture.unique()

array(['Yamaguchi', 'Niigata', 'Shizuoka', 'Yamanashi', 'Hyogo', 'Osaka',
       'Kouchi', 'Hokkaido', 'Yamagata', 'Tottori', 'Tochigi', 'Fukui',
       'Akita', 'Tokushima', 'Ohita', 'Wakayama', 'Shimane', 'Ishikawa',
       'Fukushima', 'Aomori', 'Kyoto', 'Iwate', 'Saitama', 'Gifu',
       'Nagano', 'Oakland, California', 'Shiga', 'Ehime', 'Kochi',
       'Miyagi', 'Ibaraki', 'Aichi', 'Hiroshima', 'Saga', 'Nanago',
       'Fukuoka', 'Nara', 'Domestic - San Francisco', None, 'Okayama',
       'Okinawa', 'Tokyo', 'Shizouka', 'Gunma', 'Kanagawa'], dtype=object)

In [18]:
df['description'] = df['description'].apply(lambda description: split_on_chars('WORD', description))

In [19]:
#df.description.unique()

In [20]:
df.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",Junmai Daiginjo,30.00,Yamaguchi,+3,1.4,Chunky,Pinot Noir / Chewy Whites,Ales,"Mushroom risotto, tempura, fried chicken, cavi...",This sake has an older brother Daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",Junmai Ginjo,33.00,Niigata,+1,1.6,Citrus,Sharp Whites/Dry Reds,Ales/Ambers,Suitable for light and stronger flavored dishe...,True Sake owner Beau Timken recorded Kikusui a...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",Junmai Daiginjo,49.00,Shizuoka,+/-0,1.4,Popular,Elegant reds/Complex whites,Crisp Ales,Cuisines that like big white wines.,Talk about a nice aroma package on this ultra-...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",Tokubetsu Junmai,9.00,Yamanashi,+3,1.6,Smooth,Pinot Noir/Slippery Whites,Gentle Ales,None,The nose on this awesome looking “catch cup” i...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",Honjozo,40.00,Hyogo,±0.5,1.5,Rich,Deep Reds/Fat Whites,Stouts,Perfect for pub food salty and savory and grea...,The nose on this extremely famous Honjozo from...


In [21]:
d = df.description.unique()[0]
d

'This sake has an older brother Daiginjo that has an industry leading milling rate of 23. Dassai 45 has a collection of sweet aromas including grape juice, cotton candy, and a hint of lemonade. Talk about a sake with body! This uber Daiginjo has a full-figured flavor that rushes chewy fruit tones to all corners of your mouth. It is wide and heavy with lots of expansive elements that talk to those who like a mouthful. Pay attention for a hint of anise and sneaky mild veggie aftertaste. The subtle sweetness including grape and berries becomes more pronounced when the fluid warms in the mouth.'

In [25]:
# imports
import unicodedata

# function to remove accented characters
def remove_accented_chars(text):
    clean_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return clean_text

import string

# function to remove punctuation
def remove_punctuation(text):
    clean_text = ''.join([c for c in text if c not in string.punctuation])
    return clean_text

def clean_text(text):
    clean_text = remove_punctuation(remove_accented_chars(text)).lower()
    return clean_text

In [26]:
clean_text(d)

'this sake has an older brother daiginjo that has an industry leading milling rate of 23 dassai 45 has a collection of sweet aromas including grape juice cotton candy and a hint of lemonade talk about a sake with body this uber daiginjo has a fullfigured flavor that rushes chewy fruit tones to all corners of your mouth it is wide and heavy with lots of expansive elements that talk to those who like a mouthful pay attention for a hint of anise and sneaky mild veggie aftertaste the subtle sweetness including grape and berries becomes more pronounced when the fluid warms in the mouth'

In [27]:
df.description.unique()

array(['This sake has an older brother Daiginjo that has an industry leading milling rate of 23. Dassai 45 has a collection of sweet aromas including grape juice, cotton candy, and a hint of lemonade. Talk about a sake with body! This uber Daiginjo has a full-figured flavor that rushes chewy fruit tones to all corners of your mouth. It is wide and heavy with lots of expansive elements that talk to those who like a mouthful. Pay attention for a hint of anise and sneaky mild veggie aftertaste. The subtle sweetness including grape and berries becomes more pronounced when the fluid warms in the mouth.',
       'True Sake owner Beau Timken recorded Kikusui as the first entry in his personal journal of over 200 sakes and would later purchase Kikusui as his first case of sake, thus Kikusui holds a special place on our shelves. This Ginjo has a slightly fruity nose with hints of citrus and persimmons. Kikusui has a confidently smooth beginning with a gradual thickening viscosity that ends in a

In [29]:
d = 'We used to refer to this wonderful Can as “Beau-shu” because the owner, Beau Timken, discovered this sake on a trip to Tokushima to visit this brewery. It was an exclusive sake at True Sake for the the first year in the States, but we wanted everybody to enjoy this super special sake, and today it is one, of if not the top imported unpasteurized sake. It is a signature “Genshu” or undiluted (18.5% alc) sake with fruity licks but dry tricks. The perfect “semi” sake that is semi-dry, semi-sweet, semi-bold, semi-thick, and semi-rich, but fully fun! \xa0Round and tingly. It has deep rich pear, white grape, and melon tones with a hint of citrus and rock candy. Lively, brisk and zesty. It drinks with a crisp attitude that appeals to all wine lovers. Truly one of the best selling sake in True Sake history! It was the first brew that we simply could not keep on the shelf! And if you see a can you should grab it quickly.'

In [30]:
clean_text(d)

'we used to refer to this wonderful can as beaushu because the owner beau timken discovered this sake on a trip to tokushima to visit this brewery it was an exclusive sake at true sake for the the first year in the states but we wanted everybody to enjoy this super special sake and today it is one of if not the top imported unpasteurized sake it is a signature genshu or undiluted 185 alc sake with fruity licks but dry tricks the perfect semi sake that is semidry semisweet semibold semithick and semirich but fully fun  round and tingly it has deep rich pear white grape and melon tones with a hint of citrus and rock candy lively brisk and zesty it drinks with a crisp attitude that appeals to all wine lovers truly one of the best selling sake in true sake history it was the first brew that we simply could not keep on the shelf and if you see a can you should grab it quickly'

In [32]:
re.sub('  ', ' ', clean_text(d))

'we used to refer to this wonderful can as beaushu because the owner beau timken discovered this sake on a trip to tokushima to visit this brewery it was an exclusive sake at true sake for the the first year in the states but we wanted everybody to enjoy this super special sake and today it is one of if not the top imported unpasteurized sake it is a signature genshu or undiluted 185 alc sake with fruity licks but dry tricks the perfect semi sake that is semidry semisweet semibold semithick and semirich but fully fun round and tingly it has deep rich pear white grape and melon tones with a hint of citrus and rock candy lively brisk and zesty it drinks with a crisp attitude that appeals to all wine lovers truly one of the best selling sake in true sake history it was the first brew that we simply could not keep on the shelf and if you see a can you should grab it quickly'